In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
from tensorflow.keras.layers import ZeroPadding2D, add, Input, Activation
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam , Adadelta, RMSprop, Nadam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np
import wandb
from wandb.keras import WandbCallback

In [2]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [3]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)


In [4]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)


Compute dtype: float16
Variable dtype: float32


In [5]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [6]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

In [7]:
def residual_module(data, K, stride, chanDim,
                    reg=0.0001, bnEps=2e-5, bnMom=0.9, reduce=False):
    
    shortcut = data
    
    bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(data)
    act1 = Activation("relu")(bn1)
    
    conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
                kernel_regularizer=l2(reg))(act1)
    
    
    bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                momentum=bnMom)(conv1)
    act2 = Activation("relu")(bn2)
    conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
                padding="same", use_bias=False,
    kernel_regularizer=l2(reg))(act2)
    
    
    bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                momentum=bnMom)(conv2)
    act3 = Activation("relu")(bn3)
    conv3 = Conv2D(K, (1, 1), use_bias=False,
                kernel_regularizer=l2(reg))(act3)
    
    
    
    if reduce:
        shortcut = Conv2D(K, (1, 1), strides=stride,
            use_bias=False, kernel_regularizer=l2(reg))(act1)
    
    x = add([conv3, shortcut])
    
    
    
    return x

In [8]:
def build_Resnet(width, height, depth, classes, stages, filters,
                reg=0.0001, bnEps=2e-5, bnMom=0.9):
    
    
    inputShape = (height, width, depth)
    chanDim = -1
    inputs = Input(shape=inputShape)
    x = BatchNormalization(axis=chanDim, epsilon=bnEps,
                            momentum=bnMom)(inputs)
    x = Conv2D(filters[0], (3, 3), use_bias=False,padding="same", kernel_regularizer=l2(reg))(x)
    
    
    
    for i in range(0, len(stages)):

        stride = (1, 1) if i == 0 else (2, 2)
        
        x = residual_module(x, filters[i + 1], stride,
                chanDim, reduce=True, bnEps=bnEps, bnMom=bnMom)
        
        for j in range(0, stages[i] - 1):
            
            x = residual_module(x, filters[i + 1],
                    (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)
            
    x = BatchNormalization(axis=chanDim, epsilon=bnEps,
                            momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = AveragePooling2D((8, 8))(x) 
    x = Flatten()(x)
    x = Dense(classes, kernel_regularizer=l2(reg))(x)
    x = Activation("softmax", dtype='float32')(x)
    
    model = Model(inputs, x, name="resnet")
    
    
    
    return model



In [9]:
# NUM_EPOCHS = 100
# INIT_LR = 1e-2

# def poly_decay(epoch):
#     maxEpochs = NUM_EPOCHS
#     baseLR = INIT_LR
#     power = 1.0
#     alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
#     return alpha

In [10]:
model = build_Resnet(32, 32, 3, 10, (9, 9, 9),
                                (64, 64, 128, 256), reg=0.0009)

In [11]:
Batch_size = 128
lr = 2e-2
epochs = 100

# callbacks = [LearningRateScheduler(poly_decay)]


#opt = SGD(lr= lr, momentum= 0.9, nesterov=True)
#opt = SGD(lr= lr ,decay = ,  momentum= 0.9, nesterov=True)
#
#opt = Adam(lr=lr,amsgrad=True)


s = 100 * len(trainX) // Batch_size # number of steps in 20 epochs (batch size = 32)
learning_rate0 = keras.optimizers.schedules.ExponentialDecay(lr, s, 0.96)

opt = Adam(learning_rate=lr , beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.001, amsgrad=True)



In [12]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [13]:
Resnet = model.fit(trainX, trainY, validation_data = (testX, testY)
                       , batch_size = Batch_size, epochs= epochs, verbose= 1)

Epoch 1/100
391/391 [==============================] - 28s 73ms/step - loss: 2.1474 - accuracy: 0.3445 - val_loss: 1.7975 - val_accuracy: 0.4317
Epoch 2/100
391/391 [==============================] - 28s 71ms/step - loss: 1.5404 - accuracy: 0.5249 - val_loss: 1.9806 - val_accuracy: 0.3997
Epoch 3/100
391/391 [==============================] - 28s 70ms/step - loss: 1.3072 - accuracy: 0.6115 - val_loss: 1.3610 - val_accuracy: 0.5847
Epoch 4/100
391/391 [==============================] - 28s 71ms/step - loss: 1.1692 - accuracy: 0.6633 - val_loss: 1.4448 - val_accuracy: 0.5709
Epoch 5/100
391/391 [==============================] - 28s 71ms/step - loss: 1.0787 - accuracy: 0.6919 - val_loss: 1.2361 - val_accuracy: 0.6362
Epoch 6/100
391/391 [==============================] - 27s 70ms/step - loss: 1.0066 - accuracy: 0.7157 - val_loss: 1.4657 - val_accuracy: 0.5821
Epoch 7/100
391/391 [==============================] - 28s 70ms/step - loss: 0.9483 - accuracy: 0.7357 - val_loss: 1.0495 - val_ac

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8,5))
# plt.plot(np.arange(0,31),history.history['accuracy'], color='orange')
# plt.plot(np.arange(0,31),history.history['loss'],'b')
# plt.plot(np.arange(0,31)+0.5,history.history['val_accuracy'],'r')  # offset both validation curves
# plt.plot(np.arange(0,31)+0.5,history.history['val_loss'],'g')
plt.plot(Resnet.history['accuracy'],color='orange')
plt.plot(Resnet.history['val_accuracy'],'r')
plt.plot(Resnet.history['loss'],'b')
plt.plot(Resnet.history['val_loss'],'g')
plt.legend(['Train Acc','Val Acc','Train Loss','Val Loss'])
#plt.grid()
plt.gca().set_ylim(0, 2.5) # set the vertical range to [0-1] 
plt.title("Resnet ")
plt.savefig("Resnet", dpi=300,  bbox_inches='tight')
plt.show()